# 2. Update data daily

In [13]:
from keys import TOMTOM_API_KEY, TOMTOM_API_KEY2, TOMTOM_API_KEY3
from TomTomAPI import get_traffic_map_from_grid
from HolidaysnWeather import get_weather, is_holiday
import pickle
import pandas as pd

In [2]:
coordinates = 59.34318, 18.05141 # Stockholm near Odenplan
zoom = 20

TOMTOM_keys = [TOMTOM_API_KEY, TOMTOM_API_KEY2, TOMTOM_API_KEY3]

### 2.1. Collect traffic data

In [3]:
# Getting grid variable from memory
with open('variables/grid.pickle', 'rb') as file:
    # Deserialize and retrieve grid from pickle file
    grid = pickle.load(file)

In [ ]:
# Collecting traffic flow from grid
success = False
i = 0
while i < len(TOMTOM_keys) and not success:
    try:
        traffic_map = get_traffic_map_from_grid(TOMTOM_keys[i],grid, zoom = zoom)
        success = True
    except:
        print('Failed with key:', TOMTOM_keys[i])
        i += 1
traffic_map = pd.DataFrame(traffic_map)
traffic_map

Failed with key: zJT5UBCDtRKSWwsbGp6LWjcQcIi0IcdV
num of requests: 294


In [11]:
traffic_map.coordinates = traffic_map.coordinates.apply(lambda x: str(x))
traffic_map = traffic_map.drop_duplicates()
traffic_map.coordinates = traffic_map.coordinates.apply(lambda x: eval(x))
traffic_map

/var/folders/wg/qn3bmnj948q_cdv4_z9r5xnh0000gn/T/ipykernel_71937/981005903.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traffic_map.coordinates = traffic_map.coordinates.apply(lambda x: eval(x))


,frc,currentSpeed,freeFlowSpeed,currentTravelTime,freeFlowTravelTime,confidence,roadClosure,coordinates,@version
0,FRC6,9,11,54,44,0.84,False,{'coordinate': [{'latitude': 59.34873597152101...,traffic-service-flow 1.0.120
1,FRC5,24,34,86,61,1.00,False,{'coordinate': [{'latitude': 59.33702143851012...,traffic-service-flow 1.0.120
2,FRC4,31,31,24,24,1.00,False,{'coordinate': [{'latitude': 59.34158925052537...,traffic-service-flow 1.0.120
3,FRC4,8,24,117,39,1.00,False,{'coordinate': [{'latitude': 59.34681285368509...,traffic-service-flow 1.0.120
4,FRC6,20,20,24,24,1.00,False,{'coordinate': [{'latitude': 59.34521959564415...,traffic-service-flow 1.0.120
...,...,...,...,...,...,...,...,...,...
287,FRC6,22,22,30,30,1.00,False,{'coordinate': [{'latitude': 59.34357407931478...,traffic-service-flow 1.0.120
288,FRC4,13,24,198,107,1.00,False,{'coordinate': [{'latitude': 59.33870852080807...,traffic-service-flow 1.0.120
290,FRC1,21,36,59,34,1.00,False,{'coordinate': [{'latitude': 59.34712665029686...,traffic-service-flow 1.0.120
291,FRC4,18,32,53,30,1.00,False,{'coordinate': [{'latitude': 59.35006097620218...,traffic-service-flow 1.0.120


### 2.2. Collect weather data

In [14]:
weather = get_weather(coordinates)
weather

,temperature_2m_max,temperature_2m_min,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant
2024-12-16,7.726,2.976,2.8,37.439999,273.435425


### 2.3. Collect weather data

In [15]:
holiday_status = is_holiday()
holiday_status

0

### 2.4. Joining data and uploading to Hopsworks